In [15]:
#read all files into an array (dataframes)
import numpy as np
import os
import pandas as pd
from IPython.display import display
import datetime
import re
import json
directory = os.getcwd() 

def read_data_from_files(directory):
  dataframes= []
  for root,dirs,files in os.walk(directory):
      for file in files:
         if (file.endswith("Buildingslogs.csv") & os.path.exists(file)):
            col_names = pd.read_csv(file, nrows=0).columns

            dataframes.append(pd.read_csv(file,sep=";",low_memory=False,header=[0],skiprows=[1]))
  return dataframes


#group the data by building where d[str(serial)] = group

def group_data_by_building(data):
  d = {}
  for data in data :
    for serial, group in data.groupby('serial'):
      d[str(serial)] = group
  return d


def int_to_time(input,hours_mapping):
  return datetime.datetime(2021, 1, 1) + datetime.timedelta(hours=input/hours_mapping)

def hour_rounder(t):
    # Rounds to nearest hour by adding a timedelta hour if minute >= 30
    return (t.replace(second=0, microsecond=0, minute=0, hour=t.hour)
               +datetime.timedelta(hours=t.minute//30))  

def map_to_real_time(data):
  # hou much is an hour in game time 
  # for example 1hour = 2000 game time unite
  hour_in_numbers = (int(data[0]["time"].max()) - int(data[0]["time"].min())) / (365*24)
  for df in data :
    df["time"]=df["time"].apply(int_to_time,args=[hour_in_numbers])
    df["time"]=df["time"].apply(hour_rounder)
  return data  

def read_orders_data_from_files(directory):
  dataframes= []
  for root,dirs,files in os.walk(directory):
      for file in files:
         if (file.endswith("orderslogs.csv") & os.path.exists(file)):
            col_names = pd.read_csv(file, nrows=0).columns

            dataframes.append(pd.read_csv(file,sep=";",low_memory=False,header=[0],skiprows=[1]))
  return dataframes


In [16]:
data = read_data_from_files(directory)

In [17]:
print(data[0])

        serial                        name      time  ware_economy_serial  \
0         1461     atlanteans_headquarters      1000                    2   
1         1461     atlanteans_headquarters      2000                    2   
2         1461     atlanteans_headquarters      2500                    2   
3         1461     atlanteans_headquarters      3000                    2   
4         1461     atlanteans_headquarters      4000                    2   
...        ...                         ...       ...                  ...   
385535    3917  atlanteans_foresters_house  14383651                 1498   
385536   23359         atlanteans_coalmine  14383750                 1498   
385537   23359         atlanteans_coalmine  14383750                 1498   
385538   23359         atlanteans_coalmine  14383760                 1498   
385539   23359         atlanteans_coalmine  14383760                 1498   

        worker_economy_serial  position  reserved_by_worker  \
0           

In [18]:
a=map_to_real_time(read_data_from_files(directory))

In [19]:
string=a[0].loc[(a[0]['name'] == "atlanteans_headquarters") ]["wares"][0]
print(string)
string = re.sub(r".*wares\:", "", string)

print (string)


Numberof wares:110wares:{"balsa":"0","log":"20","rope":"0","rubber":"0","granite":"50","ironwood":"0","liana":"0","water":"12","cassavaroot":"0","cocoa_beans":"0","amazons_bread":"0","chocolate":"0","fish":"3","meat":"3","ration":"0","gold_dust":"0","gold":"4","coal":"12","pick":"8","felling_ax":"0","shovel":"9","hammer":"11","machete":"0","spear_wooden":"0","chisel":"0","kitchen_tools":"0","needles":"0","stone_bowl":"0","spear_stone_tipped":"0","spear_hardened":"0","armor_wooden":"0","helmet_wooden":"0","warriors_coat":"0","tunic":"0","vest_padded":"0","protector_padded":"0","boots_sturdy":"0","boots_swift":"0","boots_hero":"0","planks":"45","spidercloth":"5","spider_silk":"9","smoked_fish":"6","smoked_meat":"6","corn":"5","cornmeal":"12","blackroot":"5","blackroot_flour":"12","atlanteans_bread":"8","quartz":"9","diamond":"7","iron_ore":"5","iron":"8","gold_ore":"0","saw":"9","milking_tongs":"2","fishing_net":"4","buckets":"2","hunting_bow":"1","hook_pole":"2","scythe":"4","bread_padd

In [20]:
Dict = json.loads(string)

In [21]:
fuck = pd.DataFrame.from_dict(Dict.items())
fuck.columns = ['ware', 'amount']

print(fuck)

             ware amount
0           balsa      0
1             log     20
2            rope      0
3          rubber      0
4         granite     50
..            ...    ...
105     spear_war      0
106  armor_helmet      0
107         armor      0
108   armor_chain      0
109  armor_gilded      0

[110 rows x 2 columns]


In [22]:
orders=read_orders_data_from_files(directory)

In [23]:
orders_maped = map_to_real_time(orders)

In [24]:
orders[0].rename(columns={'building_serial':'serial'}, inplace=True)

In [25]:
print(orders[0])

      serial                       name                time requested_Ware  \
0       1604           constructionsite 2021-01-01 11:00:00         planks   
1       1604           constructionsite 2021-01-01 11:00:00            log   
2       1604           constructionsite 2021-01-01 11:00:00    vest_padded   
3       1656                       road 2021-01-01 11:00:00        needles   
4       1604           constructionsite 2021-01-01 12:00:00            log   
...      ...                        ...                 ...            ...   
7667   38427  atlanteans_smelting_works 2022-01-01 10:00:00           coal   
7668  116262                       road 2022-01-01 10:00:00        needles   
7669   82608   atlanteans_charcoal_kiln 2022-01-01 10:00:00            log   
7670    4937    atlanteans_weaving_mill 2022-01-01 11:00:00    spider_silk   
7671    5397         atlanteans_sawmill 2022-01-01 11:00:00            log   

      amount  
0          1  
1          2  
2          1  
3  

In [26]:
merged_data = a[0].merge(orders[0], on=["serial","time"], how='left')

In [27]:
print(merged_data[~merged_data["requested_Ware"].isnull()].iloc[1,])

serial                                                                4406
name_x                                               atlanteans_spiderfarm
time                                                   2021-01-22 03:00:00
ware_economy_serial                                                     16
worker_economy_serial                                                   17
position                                                           (182,9)
reserved_by_worker                                                       0
workers                            ({4411,65535,atlanteans_spiderbreeder})
is_mine                                                                  0
is_port                                                                  0
needs_seafaring                                                          0
needs_water_ways                                                         0
passable                                                                 0
is_production_site       

In [28]:
#get one buildings stockes according to IBP Stock
#data_line is one line where wares != nan of the original data
def data_line_to_one_building_stock_IBP(data_line):
  locations = pd.DataFrame(columns=['LOCNO','LOCTYPE','NAME1','NAME2','LAND1','REGIO','ORT01','PSTLZ','PSTL2','STRAS','LOEVM','TIME_ZONE','TELF1','FABKL','KUKLA','NODETYPE','CUST_ATTR1','CUST_ATTR2','CUST_ATTR3','CUST_ATTR4','CUST_ATTR5','CUST_ATTR6','CUST_ATTR7','CUST_ATTR8','CUST_ATTR9','CUST_ATTR10','LONGITUDE','LATITUDE'])
  stocks = pd.DataFrame(columns=['MATNR','LOC_WERKS','VRFKZ','MNG01'])
  locations["LOCNO"]=data_line["position"]
  locations["LOCTYPE"]="V"
  group_data_by_building

  